### Filter VCF files 

1. Get rid of labels on the top of the file (identified by ## at the start of each line) 
2. Change file to dataframe (put all files in one big dataframe)
3. Filter by Quality 
4. Filter by Chromosome 
5. Filter by Duplicates 
6. Write to Excel File 

In [23]:
import pandas as pd 
import numpy as np 
import csv
import os 
from collections import Counter
import chardet 
os.chdir("/Users/nttur/OneDrive/Desktop/AS167_lineage")

#this does not scale well to extremely large VCF files, or very large numbers of VCF files
#if the data gets too large the overall process can take over a day to complete
#this code will take a given directory of VCF files and loop through every single one, and can create/output to an excel table
#a given number of mutation calls based on desired parameters... as is it will output ONLY UNIQUE mutations across lines
#this file can be modified to search for any number of counts of mutations, either shared or unique across any number of lines

In [24]:
#Function to create a dataframe from a vcf file 
def vcfDataFrame(path_to_vcf):
    #Read in file 
    with open (path_to_vcf, encoding='unicode_escape') as vcf_file: 
        read_vcf = csv.reader(vcf_file, delimiter='\t')

        for line in read_vcf: 
                
            # If line is part of the header, skip 
            if line[0].startswith("##"):
                pass
                
            #Discard last column name, which is the unique sample name 
            #Create empty dataframe with column names to match the VCF except for the last column 
            elif line[0].startswith("#CHROM"):
                column_names = line[0:len(line)]
                column_names[len(column_names) - 1] = "TBD"
                vcf_dF = pd.DataFrame(columns= column_names)
                path_list = path_to_vcf.split("/")
                new_column_value = path_list[len(path_list)-1].split(".")[0]
                vcf_dF["Sample_name"] = new_column_value

            else:
                #Add information in list to the dataframe 
                # vcf_dF.loc[len(vcf_dF)-1] = line[0:len(line)]
                path_list = path_to_vcf.split("/")
                new_column_value = path_list[len(path_list)-1].split(".")[0]
                vcf_line = line[0:len(line)]
                vcf_line.append(new_column_value)
                vcf_dF.loc[len(vcf_dF)+1] = vcf_line
                # vcf_dF.loc[len(vcf_dF)+1] = line[0:len(line)]
        
        # #Create a Sample_name column 
        # #Fill in Sample_name for sample name using the original VCF file name 
        # path_list = path_to_vcf.split("/")
        # new_column_value = path_list[len(path_list)-1].split(".")[0]
        # vcf_dF["Sample_name"] = new_column_value
        
        
        return vcf_dF

#Test on a single file 
a10 = vcfDataFrame("/Users/nttur/OneDrive/Desktop/AS167_lineage/167_S1.calls.vcf")
a10

,#CHROM,POS,ID,REF,ALT,QUAL,FILTER,INFO,FORMAT,TBD,Sample_name
1,NZ_CP020397.1,51,.,A,G,225.007,.,DP=252;VDB=0.274541;SGB=-0.693147;FS=0;MQ0F=0;...,GT:PL,"1:255,0",167_S1
2,NZ_CP020397.1,126,.,G,A,225.007,.,DP=246;VDB=3.54846e-07;SGB=-0.693147;FS=0;MQ0F...,GT:PL,"1:255,0",167_S1
3,NZ_CP020397.1,495,.,A,T,225.007,.,DP=244;VDB=0.461261;SGB=-0.693147;RPBZ=-0.8300...,GT:PL,"1:255,0",167_S1
4,NZ_CP020397.1,516,.,C,T,225.007,.,DP=251;VDB=0.079046;SGB=-0.693147;MQSBZ=0.8149...,GT:PL,"1:255,0",167_S1
5,NZ_CP020397.1,651,.,C,G,225.007,.,DP=245;VDB=0.844754;SGB=-0.693147;FS=0;MQ0F=0;...,GT:PL,"1:255,0",167_S1
...,...,...,...,...,...,...,...,...,...,...,...
48387,NZ_CP020399.1,549337,.,G,A,225.007,.,DP=207;VDB=0.594045;SGB=-0.693147;RPBZ=0.30572...,GT:PL,"1:255,0",167_S1
48388,NZ_CP020399.1,549431,.,G,A,225.007,.,DP=206;VDB=0.988086;SGB=-0.693147;FS=0;MQ0F=0;...,GT:PL,"1:255,0",167_S1
48389,NZ_CP020399.1,549502,.,T,C,225.007,.,DP=231;VDB=0.877285;SGB=-0.693147;RPBZ=1.56478...,GT:PL,"1:255,0",167_S1
48390,NZ_CP020399.1,549514,.,C,T,225.007,.,DP=231;VDB=0.445234;SGB=-0.693147;FS=0;MQ0F=0;...,GT:PL,"1:255,0",167_S1


In [25]:
#Implementing vcfDataFrame function for a directory of VCF files 
def create_vcf_dataFrame(path_to_files, path_to_new_excel):
    
    #Create an empty dataframe with the necessary columns to match new VCF dataframe 
    final_dF = pd.DataFrame(columns=['#CHROM', 'POS', 'ID', 'REF', 'ALT', 'QUAL', 'FILTER', 'INFO', 'FORMAT','TBD',
       'Sample_name'])
    
    #Write a new excel file with empty dataframe
    #This will be used further downstream 
    #The path to the excel file should include the new file name 
    final_dF.to_excel(path_to_new_excel)
    
    #Loop over all VCF files and use vcfDataFrame to convert to dataframes 
    #Add each new dataframe to the empty dataframe to have a dataframe including all sample info 
    for vcf in os.listdir(path_to_files):
        path_to_vcf = str(path_to_files) + "/" + str(vcf)
        new_dF = vcfDataFrame(path_to_vcf)
        final_dF = pd.concat([new_dF, final_dF])
        
    return final_dF


# tum_dF = create_vcf_dataFrame("/Users/nttur/OneDrive/Desktop/samonella stuff/vcffiles","/Users/nttur/OneDrive/Desktop/samonella stuff/vcffiles/sam_new_vcf.xlsx")
tum_dF = create_vcf_dataFrame("/Users/nttur/OneDrive/Desktop/AS167_lineage","/Users/nttur/Downloads/AS167_lineage_unique_vcf.xlsx")
tum_dF


,#CHROM,POS,ID,REF,ALT,QUAL,FILTER,INFO,FORMAT,TBD,Sample_name
1,NZ_CP020397.1,1002,.,G,A,7.79382,.,DP=1;SGB=-0.379885;FS=0;MQ0F=0;AF1=1;AC1=1;DP4...,GT:PL,"1:37,0",AS573
2,NZ_CP020397.1,1004,.,C,G,7.79382,.,DP=1;SGB=-0.379885;FS=0;MQ0F=0;AF1=1;AC1=1;DP4...,GT:PL,"1:37,0",AS573
3,NZ_CP020397.1,1233,.,C,G,225.007,.,DP=135;VDB=1.18317e-33;SGB=-0.693147;MQSBZ=0.8...,GT:PL,"1:255,0",AS573
4,NZ_CP020397.1,1266,.,G,C,225.007,.,DP=108;VDB=3.83956e-43;SGB=-0.693147;MQSBZ=0.4...,GT:PL,"1:255,0",AS573
5,NZ_CP020397.1,1267,.,C,G,225.007,.,DP=107;VDB=3.08286e-43;SGB=-0.693147;MQSBZ=0.2...,GT:PL,"1:255,0",AS573
...,...,...,...,...,...,...,...,...,...,...,...
48387,NZ_CP020399.1,549337,.,G,A,225.007,.,DP=207;VDB=0.594045;SGB=-0.693147;RPBZ=0.30572...,GT:PL,"1:255,0",167_S1
48388,NZ_CP020399.1,549431,.,G,A,225.007,.,DP=206;VDB=0.988086;SGB=-0.693147;FS=0;MQ0F=0;...,GT:PL,"1:255,0",167_S1
48389,NZ_CP020399.1,549502,.,T,C,225.007,.,DP=231;VDB=0.877285;SGB=-0.693147;RPBZ=1.56478...,GT:PL,"1:255,0",167_S1
48390,NZ_CP020399.1,549514,.,C,T,225.007,.,DP=231;VDB=0.445234;SGB=-0.693147;FS=0;MQ0F=0;...,GT:PL,"1:255,0",167_S1


In [26]:
#Filter for quality and drop duplicates 
#Discard position if quality is less than 60 
#Discard position if it is duplicated in 5 or more samples
#Put chromosome of interest as a parameter and discard all other chromosome information 
def filter_vcf_dataFrame(vcf_dataFrame, chromosome_info): 

    #Quality filter 
    vcf_dataFrame["QUAL"] = pd.to_numeric(vcf_dataFrame["QUAL"], downcast="float")
    vcf_dataFrame = vcf_dataFrame[(vcf_dataFrame["QUAL"] > 60.0)]

    #Removing unnecessary chromosomes - dependent on species 
    vcf_dataFrame = vcf_dataFrame[(vcf_dataFrame["#CHROM"] == chromosome_info)]

    #Filter for duplicates 
    #If a position is shared by 5 or more samples, discard in all samples
    
    #Make a dataframe with only unique values 
    vcf_dataFrame_nodup = vcf_dataFrame.drop_duplicates(subset=["POS"], keep = False)
    
    #Create empty list to store duplicates 
    list_of_dup = []
    
    #Iterate through dataframe, and add duplicated postions to the duplicate list 
    for index, row in vcf_dataFrame.iterrows():
        if row["POS"] not in vcf_dataFrame_nodup["POS"]: 
            list_of_dup.append(row["POS"])

    #Count the number of duplicates per position in the duplicate list  
    count_dup = Counter(list_of_dup)

    #For every position and count, if the count is greater than 5, drop the position from the dataframe
    for key, value in count_dup.items():
        if value > 1: 
            vcf_dataFrame = vcf_dataFrame.loc[vcf_dataFrame["POS"] != key]
        else: 
            pass
        
    return vcf_dataFrame


tumFiltered = filter_vcf_dataFrame(tum_dF, "NZ_CP020397.1")
tumFiltered

#Check if dropping duplicates worked: 
#Should have no values greater than or equal to 5 

# tum_check_list = tumFiltered["POS"].values.tolist()
# tum_dF_count = Counter(tum_check_list) 
# print(tum_dF_count)


,#CHROM,POS,ID,REF,ALT,QUAL,FILTER,INFO,FORMAT,TBD,Sample_name
110,NZ_CP020397.1,10640,.,A,G,99.005600,.,DP=4;VDB=0.0058656;SGB=-0.556411;FS=0;MQ0F=0;A...,GT:PL,"1:129,0",AS573
1086,NZ_CP020397.1,25783,.,A,T,93.005501,.,DP=6;VDB=0.00296109;SGB=-0.590765;FS=0;MQ0F=0;...,GT:PL,"1:123,0",AS573
1087,NZ_CP020397.1,25784,.,A,C,93.005501,.,DP=6;VDB=0.00257647;SGB=-0.590765;FS=0;MQ0F=0;...,GT:PL,"1:123,0",AS573
1088,NZ_CP020397.1,25785,.,C,G,99.005600,.,DP=5;VDB=0.00266814;SGB=-0.590765;FS=0;MQ0F=0;...,GT:PL,"1:129,0",AS573
1089,NZ_CP020397.1,25794,.,T,C,83.005402,.,DP=4;VDB=0.0058656;SGB=-0.556411;FS=0;MQ0F=0;A...,GT:PL,"1:113,0",AS573
...,...,...,...,...,...,...,...,...,...,...,...
17524,NZ_CP020397.1,2307130,.,A,C,60.005100,.,DP=4;VDB=0.0107983;SGB=-0.556411;FS=0;MQ0F=0;A...,GT:PL,"1:90,0",167_S1
18452,NZ_CP020397.1,2392874,.,CCGCGGCC,CCGCGGCCGGCGCGGCC,73.466904,.,INDEL;IDV=11;IMF=0.180328;DP=61;VDB=4.43108e-0...,GT:PL,"1:111,24,0",167_S1
23687,NZ_CP020397.1,3213581,.,T,C,67.005203,.,DP=55;VDB=4.60604e-19;SGB=-0.693145;RPBZ=-5.38...,GT:PL,"1:255,158",167_S1
23720,NZ_CP020397.1,3220775,.,ACGCGC,ACGC,75.466904,.,INDEL;IDV=59;IMF=0.302564;DP=195;VDB=0.0221621...,GT:PL,"1:113,9,0",167_S1


In [ ]:
#Write the new filtered dataframe to the empty excel sheet created above 
def createExcel(filtered_vcf, path_to_new_excel):
    
    #Create excel file with samples for each sheet 
    samples = filtered_vcf.Sample_name.unique()
    
    #Create a new sheet containing information from each sample 
    for sample_name in samples:
        sample = filtered_vcf.loc[filtered_vcf["Sample_name"] == sample_name]
        with pd.ExcelWriter(path_to_new_excel, engine="openpyxl", mode='a') as writer:  
            sample.to_excel(writer, sheet_name= sample_name )
            
# createExcel(tumFiltered, '/Users/nttur/OneDrive/Desktop/samonella stuff/vcffiles/sam_new_vcf.xlsx')
createExcel(tumFiltered, '/Users/nttur/Downloads/AS167_lineage_unique_vcf.xlsx')